In [ ]:
noFrameAck = 0
m=3
S_window = 2**(m-1)
currentWindow = [None for i in range(S_window)]
Sf = 0 # Sequence number of first frame in window
Sn = 0 # Sequence number of Nth frame window
Sn_Ws =0
data = ['A','B','C','D','E','F','G','H','I']

In [ ]:
def checkAck(currentFrame):
    global noFrameAck,Sf,Sn,Sn_Ws,data
    currentFrameSeqNo = 0
    
    while(True):
        print('Buffer',data[noFrameAck:S_window+noFrameAck])
        print('Avaiable Frames in Current Window For ACK',currentFrame)
        if(currentFrameSeqNo == len(currentFrame)):
            print('All Frames ACK in current Window')
            Sf = noFrameAck
            Sn += 1
            Sn_Ws +=1 
            return True
        print('Waiting to ACK for Frame',currentFrame[currentFrameSeqNo])
        frameType = input('Enter ACK or NACK \n')
        if(frameType == 'C'):
            break
        if(frameType == 'ACK'):
            
            print(f'Frame {currentFrame[currentFrameSeqNo]} has ACK')
            currentFrameSeqNo +=1
            noFrameAck += 1
        else:
            print(f'Frame {currentFrame[currentFrameSeqNo]} has NACK')
            currentFrameSeqNo +=1
            return 'NACK'


In [ ]:
frame_S=None; # frame to be sent
frame_T=None; #
while(True):
        
        if (Sn) >= len(data):
            break
        else:
            if Sn == 0:
                print('Data Sent',data[Sf:Sn_Ws+S_window])
                Sn = S_window-1
            if checkAck(data[Sf:Sn_Ws+S_window]): # Check for individual ACK for Each Frame
                pass
            else:
                break
                
#                 print('Next data to be sent',data[Sn])

            
        
    

In [94]:
from time import sleep
from IPython.display import display, Markdown, Latex
data = ['A','B','C','D','E','F','G','H','I']
status = ['ACK','NACK','NACK','ACK','NACK','NACK','ACK','NACK','ACK']
data = [{'val':val,'status':status} for val,status in zip(data,status)]
window_size = 4

class Recevier:
    def __init__(self, isReady,data):
        self.isReady = isReady
        self.data = data
        self.current_frames = []
    def check_ready(self):
        print("Checking for availablity")
        sleep(3)
        return True
    def check_frames(self,current_frame,batch):
        if(batch % 4 == 0):
            print('Received',[x['val'] for x in current_frame])
            self.current_frames = current_frame
            return True
    def recv_status(self):
        for i in range(len(self.current_frames)):
            sleep(1.3)
            print(f"Frame {self.current_frames[i]['val']} is {self.current_frames[i]['status']}")
            if self.current_frames[i]['status'] == 'NACK':
                print(f"Resending frame {self.current_frames[i]['val']}")
                sleep(3)
                self.current_frames[i]['status'] = 'ACK'
                print(f"Frame {self.current_frames[i]['val']} is {self.current_frames[i]['status']}")
        status_list = [x['status'] for x in self.current_frames]
        result = status_list.count(status_list[0]) == len(status_list)
        if result:
            print('All Okay')
            return True

    
class Sender:
    def __init__(self,data,window_size):
        self.data = data
        self.seq_no = 0
        self.window_size =window_size
        self.sn = 0
    def send_frames(self,batch):
        while self.seq_no < len(data): #also check if all are send or not
            if(self.seq_no % 4 == 0 ):
                for i in [x['val'] for x in self.data[0+batch:self.window_size+batch]]:
                    sleep(2)
                    print(f"Sending Frame {i}")
                return True,self.data[0+batch:self.window_size+batch]
            self.seq_no += 1
        return False,self.data[0+batch:self.window_size+batch]

In [95]:
recevier = Recevier(False,data)
sender = Sender(data,4)

def testing(data):
    print('Data in current window',[x['val'] for x in data[0:4]]);
    batch = 0
    while batch < len(data):
        if recevier.check_ready():
            print('Ready')
            display(Markdown('# Sender Side'))
            status,frames = sender.send_frames(batch)
            if(status):
                display(Markdown('# Receiver Side'))
                if(recevier.check_frames(frames,batch)):
                    if(recevier.recv_status()):
                        batch += 4
        print("Done")
testing(data)

Data in current window ['A', 'B', 'C', 'D']
Checking for availablity
Ready


# Sender Side

Sending Frame A
Sending Frame B
Sending Frame C
Sending Frame D


# Receiver Side

Received ['A', 'B', 'C', 'D']
Frame A is ACK
Frame B is NACK
Resending frame B
Frame B is ACK
Frame C is NACK
Resending frame C
Frame C is ACK
Frame D is ACK
All Okay
Done
Checking for availablity
Ready


...

Received ['E', 'F', 'G', 'H']
Frame E is NACK
Resending frame E
Frame E is ACK
Frame F is NACK
Resending frame F
Frame F is ACK
Frame G is ACK
Frame H is NACK
Resending frame H
Frame H is ACK
All Okay
Done
Checking for availablity
Ready


# Sender Side

Sending Frame I


# Receiver Side

Received ['I']
Frame I is ACK
All Okay
Done
